### **Install and Imports**

In [ ]:
# !pip install numpy emoji langdetect contractions beautifulsoup4 pandas nltk


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup # BeautifulSoup for parsing HTML and XML documents (Richardson, 2025)
import re # Regular expression library for text processing
import emoji
import textwrap # textwrap library for formatting text
from collections import Counter
import pandas as pd
# NLP processing libraries # (NLTK Project, 2024)
# Natural Language Toolkit (NLTK) is a suite of libraries and programs for
# statistical natural language processing (NLP) 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import WordNetLemmatizer
import contractions 
from langdetect import detect, detect_langs, DetectorFactory
from collections import Counter
from emoji import EMOJI_DATA
from concurrent.futures import ProcessPoolExecutor

### **Read from CSV file: 2024**


In [2]:
# show full column content
pd.set_option("display.max_colwidth", None)  

#### **Data Clean**

In [3]:
#df = pd.read_csv("output/2024_LD_1.csv")
df = pd.read_csv("Raw Data/2024_Sample_Complete.csv")
# df = pd.read_csv("output/2024_Sample_Complete_2.csv")
df.head(1)

,text
0,@iFightForKids What if it was actually biden biting the secret service agents and they just blamed the dogs instead?


In [4]:
#rename cloumn text to tweet 
df = df.rename(columns={'text': 'tweet'})

### **Duplicates, Null and Length Handling**





In [5]:
duplicates= df[df.duplicated(subset=['tweet'], keep=False)]
print("Number of duplicate tweet rows:", len(duplicates))
df = df.drop_duplicates(subset=['tweet'], keep='first').reset_index(drop=True)
print("Number of rows after removing duplicates:", len(df))

Number of duplicate tweet rows: 291477
Number of rows after removing duplicates: 581780


In [6]:
# # for 2024: minimize to 4000 rows 
# df = df.iloc[:6000]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581780 entries, 0 to 581779
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   581780 non-null  object
dtypes: object(1)
memory usage: 4.4+ MB


**Handle Null Values**

In [7]:
print("Num of null values in DataFrame:")
# pandas isnull() method to check for null values in the DataFrame
# The null values will help to identify any missing data that may need to be addressed before analysis and machine learning tasks
df.isnull().sum()
df = df.dropna(subset=['tweet']).reset_index(drop=True)
print("Number of rows after removing null tweet rows:", len(df))

Num of null values in DataFrame:
Number of rows after removing null tweet rows: 581780


**Handle Text Lenghth**

In [8]:
# remove tweets with less than 2 words to ensure meaningful analysis
# Initialize text_list from the 'tweet' column in the DataFrame
text_list = df['tweet'].tolist() 
text_list = sorted(text_list, key=lambda x: len(str(x)))

# Display top 5 shortest tweets
print("Top 5 shortest tweets:")
for tweet in text_list[:5]:
    wrap_text = textwrap.fill(str(tweet), width=80)
    print(wrap_text)
    print("-" * 50)
# drop tweets with with just two words from dataframe
df = df[df['tweet'].str.split().str.len() > 2]
# Display the shape of the DataFrame after dropping short tweets
print("Shape of the DataFrame after dropping short tweets:", df.shape) 

Top 5 shortest tweets:
💯
--------------------------------------------------
Q
--------------------------------------------------
😮
--------------------------------------------------
🤮
--------------------------------------------------
💙
--------------------------------------------------
Shape of the DataFrame after dropping short tweets: (573089, 1)


## **Normilazation**


**Normalize whitespace (\n → space)**

In [9]:
df['tweet']= df['tweet'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [10]:
df.head()

,tweet
0,@iFightForKids What if it was actually biden biting the secret service agents and they just blamed the dogs instead?
1,"Welcome to Kamala Harris' America. Where Islamic Terrorists are allowed to cross the border, buy guns, them murder innocent people while shouting ""allahu akbar"". #KamalaDumpsterFire #Kamala Surge #DemocratsHateAmerica #VoteTrumpToSaveAmerica #DeportThemAll"
2,"@Penelopesire1 @kylenabecker I was just talking to my wife about that, when Kamala Harris said she wouldn’t change anything in the past 3 1/2 years versus Trump telling all about the mistakes made his 1st term and won’t make them again, that’s apparent, thank God 🙏"
3,@BarackObama @vicblends I would take Donald Trump‘s family values and the way he governs over Kamala any day.
4,NFL legend gives angry response to claims he endorsed Kamala Harris👏👏 https://t.co/0BFG0FnRtV


**Ellipsis, comma and quotations**
- (…, U+2026) into three dots (...)

In [11]:
def normalizeText(text):
    if not isinstance(text, str):
        return text
    
    # Normalize ellipsis (…) to "..."
    text = text.replace("…", "...")
    
    # Normalize curly quotes to straight quotes
    text = text.replace("“", '"').replace("”", '"')
    text = text.replace("‘", "'").replace("’", "'")
    
    # Normalize en-dash and em-dash to plain dash
    text = text.replace("–", "-").replace("—", "-")
    
    # Collapse multiple dots into three max (avoid "......")
    text = re.sub(r"\.{3,}", "...", text)
    
    return text


In [12]:
df['tweet']= df['tweet'].apply(normalizeText)

**Handle Non-English Tweets**


In [ ]:
# DetectorFactory.seed = 0  # deterministic results

# # Regex patterns
# urlPattern = re.compile(r'https?://\S+')
# mentionPattern = re.compile(r'@\w+')

# # Remove URLs, mentions, and emojis to evaluate language
# def removeNoise(text: str):    
#     t = urlPattern.sub(' ', str(text))
#     t = mentionPattern.sub(' ', t)

#     # Count hashtags but keep them in the text
#     numHash = t.count('#')

#     # Remove emojis only
#     cleanTweet = ''.join(ch if ch not in EMOJI_DATA else ' ' for ch in t)

#     return cleanTweet.strip(), numHash

# # Remove text that only contains hashtags or is not in English
# def removeLanguage(df, col="text", verbose=True):
#     detected = []
#     numRemoved = []
#      # check each row
#     def validate(row_text):
#         text, numHash = removeNoise(row_text)
#         words = text.split()

#         # Filter hashtag-only tweets (>=3 hashtags but very few words)
#         if numHash >= 3 and len(words) < 4:
#             if verbose:
#                 print(f"Where Just Hashtag Tweets: {row_text}")
#             detected.append("hashtag_only")
#             numRemoved.append(row_text)
#             return False

#         try:
#             langs = detect_langs(text or "a")
#             best = langs[0]
#             lang, prob = best.lang, best.prob
#         except Exception:
#             lang, prob = "error", 0.0

#         detected.append(lang)

#         if lang == "en" and prob >= 0.55:
#             return True
#         else:
#             if verbose:
#                 print(f"Where Non-English Tweets({lang} {prob:.2f}): {row_text}")
#             numRemoved.append(row_text)
#             return False

#     mask = df[col].apply(validate)
#     filtered = df[mask].copy()

#     if verbose:
#         print("\n Summary of Language Found:")
#         for k, v in Counter(detected).most_common():
#             print(f"{k}: {v}")
#         print(f"\nRemaining rows: {len(filtered)} / {len(df)}")

#     return filtered

In [13]:
DetectorFactory.seed = 0  # keep deterministic results

urlPattern = re.compile(r'https?://\S+')
mentionPattern = re.compile(r'@\w+')

def pre_clean(series: pd.Series):
    cleaned = series.astype(str)
    cleaned = cleaned.str.replace(urlPattern, ' ', regex=True)
    cleaned = cleaned.str.replace(mentionPattern, ' ', regex=True)
    num_hash = cleaned.str.count('#')
    cleaned = cleaned.apply(lambda s: ''.join(' ' if ch in EMOJI_DATA else ch for ch in s).strip())
    return cleaned, num_hash

def detect_lang_safe(text):
    try:
        return detect_langs(text or "a")[0]
    except Exception:
        return None

def removeLanguage(df, col="text", verbose=True, min_prob=0.55, workers=None):
    cleaned, num_hash = pre_clean(df[col])

    mask = pd.Series(True, index=df.index)
    mask &= ~((num_hash >= 3) & (cleaned.str.split().str.len() < 4))

    subset = cleaned[mask]

    cache = {}
    def detect_with_cache(text):
        if text in cache:
            return cache[text]
        best = detect_lang_safe(text)
        cache[text] = best
        return best

    if workers and len(subset) > 0:
        def detect_batch(chunk):
            DetectorFactory.seed = 0
            return [detect_with_cache(text) for text in chunk]
        with ProcessPoolExecutor(max_workers=workers) as pool:
            results = sum(pool.map(detect_batch, np.array_split(subset.values, workers)), [])
    else:
        results = [detect_with_cache(text) for text in subset.values]

    detected = []
    keep = pd.Series(False, index=subset.index)
    for idx, best in zip(subset.index, results):
        if best is None:
            detected.append(("error", 0.0))
            continue
        lang, prob = best.lang, best.prob
        detected.append((lang, prob))
        if lang == "en" and prob >= min_prob:
            keep.at[idx] = True

    filtered = df.loc[keep.reindex(df.index, fill_value=False)].copy()

    if verbose:
        tallies = Counter(lang for lang, _ in detected)
        print("\nSummary of Language Found:")
        for lang, count in tallies.most_common():
            print(f"{lang}: {count}")
        print(f"\nRemaining rows: {len(filtered)} / {len(df)}")

    return filtered

In [14]:
#remove if not english and not hashtag only
df = removeLanguage(df, col="tweet", verbose=False)
# remove from df "tweet" column so that index that are english and not hashtag only are kept
df = df[df['tweet'].notna()].reset_index(drop=True)


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481427 entries, 0 to 481426
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   481427 non-null  object
dtypes: object(1)
memory usage: 3.7+ MB


In [16]:
df.head(5)

,tweet
0,@iFightForKids What if it was actually biden biting the secret service agents and they just blamed the dogs instead?
1,"Welcome to Kamala Harris' America. Where Islamic Terrorists are allowed to cross the border, buy guns, them murder innocent people while shouting ""allahu akbar"". #KamalaDumpsterFire #Kamala Surge #DemocratsHateAmerica #VoteTrumpToSaveAmerica #DeportThemAll"
2,"@Penelopesire1 @kylenabecker I was just talking to my wife about that, when Kamala Harris said she wouldn't change anything in the past 3 1/2 years versus Trump telling all about the mistakes made his 1st term and won't make them again, that's apparent, thank God 🙏"
3,@BarackObama @vicblends I would take Donald Trump's family values and the way he governs over Kamala any day.
4,NFL legend gives angry response to claims he endorsed Kamala Harris👏👏 https://t.co/0BFG0FnRtV


### **Save to CSV**

In [17]:
df.to_csv('2024_Sample_Complete_Clean.csv', index=False)

Deepak, 2017. nltk - wordnet lemmatization and pos tagging in python - Stack Overflow. [online] 
Available at: <https://stackoverflow.com/questions/15586721/wordnet-lemmatization-and-pos-tagging-in-python> [Accessed 26 June 2025].
Dutta, T., 2024. Sentiment Analysis of Customer Reviews: A Guide for Beginners | by Tanushree Dutta | Medium. [online] 
Available at: <https://medium.com/@14duttatanushreeya/sentiment-analysis-of-customer-reviews-a-guide-for-beginners-4a5f2194a121> 
GeeksforGeeks, 2024a. How to Remove HTML Tags from String in Python - GeeksforGeeks. [online] Available at: <https://www.geeksforgeeks.org/python/how-to-remove-html-tags-from-string-in-python/> [Accessed 25 June 2025].
GeeksforGeeks, 2024b. Python | Lemmatization with NLTK - GeeksforGeeks. [online] Available at: <https://www.geeksforgeeks.org/python-lemmatization-with-nltk/> [Accessed 26 June 2025].
Lane, K., 2021. Tokenization with NLTK. When it comes to NLP, tokenization is a… | by Kelsey Lane | Medium. [online] Available at: <https://medium.com/%40kelsklane/tokenization-with-nltk-52cd7b88c7d> [Accessed 26 June 2025].
NLTK Project, 2024. NLTK :: Natural Language Toolkit. [online] 
Available at: <https://www.nltk.org/> [Accessed 27 June 2025].
Richardson, L., 2025. Beautiful Soup Documentation — Beautiful Soup 4.13.0 documentation. [online] 
Available at: <https://www.crummy.com/software/BeautifulSoup/bs4/doc/> [Accessed 27 June 2025].